# 可以用线程来执行阻塞式I/O，但不要用它做平行计算

In [1]:
import logging
from pprint import pprint

Python采用GIL（全局解释器锁）机制来确保这种一致性。

GIL有一种非常显著的负面影响，用C++或Java等语言写程序时，可以同时执行多条线程，以充分利用计算机所配备的多个CPU核心。Python在同一时刻，只有一条线程可以向前执行。

**示例：**用Python执行一项计算量很大的任务。编写了一种非常原始的因数分解算法。

In [2]:
def factorize(number):
    for i in range(1, number + 1):
        if number % i == 0:
            yield i

In [3]:
from time import time
numbers = [2139079, 1214759, 1516637, 1852285]
start = time()
for number in numbers:
    list(factorize(number))
end = time()
print('Took %.3f seconds' % (end - start))

Took 1.436 seconds


In [4]:
from threading import Thread

class FactorizeThread(Thread):
    def __init__(self, number):
        super().__init__()
        self.number = number

    def run(self):
        self.factors = list(factorize(self.number))

为了实现平行计算，为numbers列表中的每个数字，都启动一条线程。

In [5]:
start = time()
threads = []
for number in numbers:
    thread = FactorizeThread(number)
    thread.start()
    threads.append(thread)

In [6]:
for thread in threads:
    thread.join()
end = time()
print('Took %.3f seconds' % (end - start))

Took 1.438 seconds


Python支持多线程的理由：  
1. 借助多线程，能够令Python程序自动以一种看似平行的方式，来执行这些函数。因为CPython在执行Python线程时，可以保证一定程度的公平。
2. 处理阻塞式的I/O操作，Python在执行某些系统调用事，会触发此类操作。为了响应阻塞式请求，可以借助线程，把程序与耗时的I/O操作隔离开。

**示例：**通过串行端口发送信号，以便远程控制一架直升飞机。采用一个速度较慢的系统调用来模拟这项活动。

In [7]:
# 该函数请求操作系统阻塞0.1秒，然后把控制权还给程序。
import select, socket

# Creating the socket is specifically to support Windows. Windows can't do
# a select call with an empty list.
def slow_systemcall():
    select.select([socket.socket()], [], [], 0.1)

In [8]:
start = time()
for _ in range(5):
    slow_systemcall()
end = time()
print('Took %.3f seconds' % (end - start))

Took 0.548 seconds


**问题：**主程序在运行slow_systemcall函数的时候，不能继续向下执行，系统主线程会卡在select系统调用那里。

In [9]:
start = time()
threads = []
for _ in range(5):
    thread = Thread(target=slow_systemcall)
    thread.start()
    threads.append(thread)

In [10]:
def compute_helicopter_location(index):
    pass

for i in range(5):
    compute_helicopter_location(i)
for thread in threads:
    thread.join()
end = time()
print('Took %.3f seconds' % (end - start))

Took 0.192 seconds
